In [1]:
%matplotlib inline
# This command is to plot images in the notebook

In [1]:
from __future__ import print_function
from fenics import *
import time
import matplotlib.pyplot as plt

In [2]:
T = 1.0        # final time
num_steps = 40     # number of time steps
dt = T / num_steps # time step size

# Create mesh and define function space
nx = ny = 30
#mesh = RectangleMesh(Point(-2, -2), Point(2, 2), nx, ny)
mesh = Mesh("rocke2.xml")
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
V = FunctionSpace(mesh, P1)

In [3]:
# Define initial value
#u_0 = Expression('2*exp(-a*pow(x[0]-13.0, 2) - a*pow(x[1]-13.0, 2))+exp(-a*pow(x[0]-10.0, 2) - a*pow(x[1]-10.0, 2))+exp(-a*pow(x[0]-48.0, 2) - a*pow(x[1]-42.0, 2))+exp(-a*pow(x[0]-15.0, 2) - a*pow(x[1]-15.0, 2))+exp(-a*pow(x[0]-25.0, 2) - a*pow(x[1]-10.0, 2))+exp(-a*pow(x[0]-40.0, 2) - a*pow(x[1]-40.0, 2))+exp(-a*pow(x[0]-20.0, 2) - a*pow(x[1]-6.0, 2))+exp(-a*pow(x[0]-20.0, 2) - a*pow(x[1]-5.0, 2))+exp(-a*pow(x[0]-12.5, 2) - a*pow(x[1]-11.0, 2))+exp(-a*pow(x[0]-24.0, 2) - a*pow(x[1]-11.0, 2))+exp(-a*pow(x[0]-20.0, 2) - a*pow(x[1]-25.0, 2))+exp(-a*pow(x[0]-20.0, 2) - a*pow(x[1]-26.0, 2))+exp(-a*pow(x[0]-30.0, 2) - a*pow(x[1]-25.0, 2))+exp(-a*pow(x[0]-30.0, 2) - a*pow(x[1]-30.0, 2))',
#                 degree=2, a=10)
u_0 = Expression(('exp(-a*pow(x[0], 2) - a*pow(x[1], 2))+exp(-a*pow(x[0]-1.0, 2) - a*pow(x[1]-1.0, 2))'),
                 degree=2, a=5)
u_n = interpolate(u_0, V)
n = FacetNormal(mesh)
x_0_e = Expression('x[0]',
                 degree=2)
x_1_e = Expression('x[1]',
                 degree=2)
x_0 = interpolate(x_0_e, V)
x_1 = interpolate(x_1_e, V)
# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
plot(u_n,scalarbar = True)
plt.show()

In [4]:
a =  u*v*dx + dt*dot(grad(u), grad(v))*dx
L = u_n*v*dx
# Create VTK file for saving solution
vtkfile = File('heat_gaussian_neumman/solution.pvd')

In [5]:
# Time-stepping
u = Function(V)
t = 0
integral_ini_1 = assemble(u_n*dx)
integral_ini_2 = assemble(x_0*u_n*dx)
integral_ini_3 = assemble(x_1*u_n*dx)
print("mass: ",integral_ini_1)
print("mean x: ",integral_ini_2)
print("mean y: ",integral_ini_3)

mass:  1.255499073822398
mean x:  0.6265531501454454
mean y:  0.6265531501454459


In [6]:
#Solution
print("iteration: 0")
plot(u_n,scalarbar = True)
plt.show()
meansx = [0.0]
meansy = [0.0]
masses = [0.0]
time = [0.0]
for n in range(num_steps):

    # Update current time
    t += dt
    time.append(t)
    # Compute solution
    solve(a == L, u)
    integral_1 = assemble(u*dx)
    integral_2 = assemble(x_0*u*dx)
    integral_3 = assemble(x_1*u*dx)
    print("mass: ",integral_1)
    masses.append(abs(integral_1-integral_ini_1))
    print("mean x: ",integral_2)
    meansx.append(abs(integral_2-integral_ini_2))
    print("mean y: ",integral_3)
    meansy.append(abs(integral_3-integral_ini_3))
    print("iteration: ",n+1)
    # Save to file and plot solution
    vtkfile << (u, t)
    plot(u,scalarbar = True)
    plt.show()
    # Update previous solution
    u_n.assign(u)

iteration: 0
mass:  1.2554990738223968
mean x:  0.6262470028666419
mean y:  0.6262470028666425
iteration:  1
mass:  1.2554990738223948
mean x:  0.6256805256229365
mean y:  0.6256805256229366
iteration:  2
mass:  1.2554990738223932
mean x:  0.6248395945819527
mean y:  0.6248395945819523
iteration:  3
mass:  1.2554990738223943
mean x:  0.6237230362650299
mean y:  0.6237230362650305
iteration:  4
mass:  1.255499073822393
mean x:  0.6223394714436898
mean y:  0.6223394714436901
iteration:  5
mass:  1.25549907382239
mean x:  0.6207036498515044
mean y:  0.6207036498515048
iteration:  6
mass:  1.2554990738223888
mean x:  0.6188336272071521
mean y:  0.6188336272071524
iteration:  7
mass:  1.2554990738223895
mean x:  0.6167488243524356
mean y:  0.6167488243524358
iteration:  8
mass:  1.2554990738223872
mean x:  0.6144687760773174
mean y:  0.6144687760773168
iteration:  9
mass:  1.255499073822388
mean x:  0.6120123706828272
mean y:  0.6120123706828272
iteration:  10
mass:  1.2554990738223875
mean

In [7]:
plt.axes(xlim=(0, T),ylim=(-max(max(meansx),max(meansy)), max(max(meansx),max(meansy))))
plt.ylabel('| mass(t) - mass(0) |',fontsize=20)
plt.xlabel('Time',fontsize=20)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
#plt.title("Mass conservation Neumann B.C.",fontsize=20)
plt.plot(time,masses,color='b', linewidth=2.0) # if mass is conserved the plot should be zero
plt.show()

plt.axes(xlim=(0, T),ylim=(-max(max(meansx),max(meansy)), max(max(meansx),max(meansy))))
plt.ylabel('| x-mean(t) - x-mean(0) |',fontsize=20)
plt.xlabel('Time',fontsize=20)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
#plt.title("Mean conservation Neumann B.C.",fontsize=20)
plt.plot(time,meansx, color='b', linewidth=2.0) # if mean is conserved the plot should be zero
plt.show()

plt.axes(xlim=(0, T),ylim=(-max(max(meansx),max(meansy)), max(max(meansx),max(meansy))))
plt.ylabel('| y-mean(t) - y-mean(0) |',fontsize=20)
plt.xlabel('Time',fontsize=20)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
#plt.title("Mean conservation Neumann B.C.",fontsize=20)
plt.plot(time,meansy, color='b', linewidth=2.0) # if mean is conserved the plot should be zero
plt.show()